In [1]:
import os
import cv2
from shutil import copyfile
import matplotlib.pyplot as plt
import csv
import random
import cv2
import numpy as np
from skimage import data
from skimage import exposure
from skimage.transform import match_histograms
from models import *
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
cuda = True if torch.cuda.is_available() else False
G_AB = GeneratorResNet(res_blocks=9)
if cuda:
    G_AB = G_AB.cuda()
G_AB.load_state_dict(torch.load('/media/arg_ws3/5E703E3A703E18EB/research/cycle_box_sim2real/saved_models/box_sim2real_G_LAMDAID_1/G_AB_1031525_.pth'))
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

data_transform = transforms.Compose(
                [ transforms.Resize((240, 320), Image.BICUBIC),
                #transforms.RandomCrop((opt.img_height, opt.img_width)),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
    )

In [3]:
def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.

    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.

    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_

    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.unsqueeze(0)
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.unsqueeze(0)

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new_full((3, height * ymaps + padding, width * xmaps + padding), pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid

In [4]:
def sample_images(path):
    """Saves a generated sample from the test set"""
    #imgs = next(iter(val_dataloader))
    #image = cv2.imread("/media/arg_ws3/5E703E3A703E18EB/data/mm_unity/unity_boxes/box_108/Scene65/11_original.png")
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.open(path)
    #pil_im = Image.fromarray(image)
    pil_im = data_transform(image)
    pil_im = pil_im.unsqueeze(0)

    #image = cv2.resize(image, (256, 256)) 
    #image = np.transpose(image, (2, 0, 1))
    #image = torch.tensor(image)
    #image = image.unsqueeze(0)

    #print(image.shape)
    #print(pil_im.shape)

    my_img = Variable(pil_im.type(Tensor))
    my_img_fake = G_AB(my_img)
    my_img_fake = my_img_fake.squeeze(0).detach().cpu()
    
    grid = make_grid(my_img_fake, nrow=5, normalize=True)
    pil_ = grid.mul_(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
    #print(pil_)
    pil_ = np.array(pil_)
    pil_ = pil_[...,::-1]
    pil_ = cv2.resize(pil_, (640, 480))
    return pil_
    #print(pil_.dtype)
    #cv2.imwrite("crop_mask1.jpg", pil_)

    #save_image(my_img_fake, 'boxxx.png', nrow=5, normalize=True)

In [5]:
root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/box_gan_LAMDAID_1/image/"
image_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/image/"
mask_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/mask/"
box_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_FCN/unity/box/"
box_list = os.listdir(box_root)

In [6]:
for box in box_list:
    mask_name = box.split('_original.png')[0] + '_seg.png'
    #box_img = cv2.imread(os.path.join(box_root, box))
    matched = sample_images(os.path.join(box_root, box))
    image = cv2.imread(os.path.join(image_root, box))
    mask_img = cv2.imread(os.path.join(mask_root, mask_name), 0)
    mask_img[mask_img == 0] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    matched = cv2.bitwise_and(matched, mask, mask=mask_img)
    mask_img[mask_img < 255] = 1
    mask_img[mask_img == 255] = 0
    mask_img[mask_img == 1] = 255
    mask = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)
    image = cv2.bitwise_and(image, mask, mask=mask_img)
    result = cv2.add(matched, image)
    cv2.imwrite(os.path.join(root, box), result)